## Prelude data

The notebook requires following prerequisites:
```
Pkg.update()
Pkg.add("CSV")
```

In [9]:
using CSV

In [3]:
π * 3

9.42477796076938

In [55]:
# Read in data sources
function readCSVData(path)
    return CSV.read(path; types=[Float32,Float32])
end

data = readCSVData("data.csv")
data[1:4, :]

,2.1014188538510608,-1.2885704654058074
1,Nullable{Float32}(-2.62788),Nullable{Float32}(-5.881)
2,Nullable{Float32}(2.81439),Nullable{Float32}(2.72146)
3,Nullable{Float32}(2.67629),Nullable{Float32}(-1.2082)
4,Nullable{Float32}(3.43626),Nullable{Float32}(3.44076)


In [52]:
mean(data[:,1])

LoadError: MethodError: no method matching /(::Nullable{Float64}, ::Int64)[0m
Closest candidates are:
  /([1m[31m::BigFloat[0m, ::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:267
  /([1m[31m::DataArrays.NAtype[0m, ::Number) at /Users/pataoengineer/.julia/v0.5/DataArrays/src/operators.jl:631
  /([1m[31m::Integer[0m, ::Integer) at int.jl:35
  ...[0m